In [78]:
# # dependency installation
# !pip install typing_extensions==4.6.1 pydantic pydantic-core cuml umap-learn pandas numpy matplotlib scikit-learn nltk langdetect bertopic[spacy] openai
# !pip install bertopic flair

In [23]:
# Load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import download
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import json
from nltk.stem import WordNetLemmatizer
import os

from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from wordcloud import WordCloud
import umap
import matplotlib.pyplot as plt
import openai
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from bertopic import BERTopic  
from sklearn.metrics import silhouette_score
from nltk.tokenize import sent_tokenize, word_tokenize

#find stop words, along with recipe realted stopwords
stop = stopwords.words('english')

FinancialLabel = "Financial/Job Problem"
Legallabel = "Legal Problem"
SchoolLabel = "School or Academic Related Problem"
HealthLabel = "Lack of Access to Health/Mental Health Care"
FinancialPredicted = "Financial - Predicted"
LegalPredicted = "Legal - Predicted"
SchoolPredicted = "School - Predicted"
HealthPredicted = "Health - Predicted"


## Topic Modeling

In [24]:
# Pile all comments into single array

fix = FinancialLabel
col = FinancialPredicted

data = pd.read_csv('final working dataset with only suicide articles.csv')
data = data[data[fix] > 0]
comments_all = data['Only Reason']
comments_all = comments_all.dropna()

In [25]:
comments_all = comments_all.to_list()
len(comments_all)

810

In [26]:
REDUCE_FREQUENT_WORDS = True
STOP_WORDS_FLAG = True # stops words being used in topics, but not for embeddings
NGRAM_RANGE = (1,2)  
PRETRAINED_SENTENCE_MODEL = 'all-mpnet-base-v2'

In [27]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [28]:
sentence_model = SentenceTransformer(PRETRAINED_SENTENCE_MODEL)
embeddings = sentence_model.encode(comments_all, show_progress_bar=True)

Batches: 100%|██████████| 26/26 [00:26<00:00,  1.04s/it]


In [29]:
# init model
stopwords = list(stopwords.words('english'))
umap_model = umap.UMAP(n_neighbors=10, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=NGRAM_RANGE)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=REDUCE_FREQUENT_WORDS)
representation_model = KeyBERTInspired()

In [8]:
#for siz in range(6, 7):
model = BERTopic(top_n_words = 10,
                    n_gram_range = (1,2),
                    language="english",
                    min_topic_size=6,
                    nr_topics=20,
                    calculate_probabilities = True,
                    embedding_model = PRETRAINED_SENTENCE_MODEL,
                    representation_model=representation_model,
                    vectorizer_model=vectorizer_model)

topics, probs = model.fit_transform(comments_all, embeddings)

cleaned_docs = model._preprocess_text(comments_all)
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topicsForCS = model.get_topics()
topicsForCS.pop(-1, None)
topic_words = [[word for word, _ in model.get_topic(topic) if word != ''] for topic in topicsForCS]
#topic_words = [[words for words, _ in model.get_topic(topic)] for topic in range(len(set(topicsForCS))-1)]
final_topic_words = []
for i in topic_words:
    if len(i) > 0:
        final_topic_words.append(i)

# Evaluate
coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_v')
coherencev = coherence_model.get_coherence()

coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_npmi')
coherence = coherence_model.get_coherence()
# Calculate embeddings for your documents  
embeddings1 = model._extract_embeddings(comments_all)  
# Calculate Silhouette score  
score = silhouette_score(embeddings1, topics)  
print(" CV score: ",coherencev)
print(" C NPMI score: ",coherence)
print(" Silhouette score: ",score)
print(" Number of topics: ",len(set(topics))-1)

tit = "All - " + fix
fig = model.visualize_barchart(top_n_topics=100, title = tit)
fig


 CV score:  0.5260781497810323
 C NPMI score:  -0.2314219522081835
 Silhouette score:  0.06988875
 Number of topics:  19


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4280146062374115, 0.4995546340942383, 0.5116849541664124,
                    0.5899918675422668, 0.602181077003479],
              'xaxis': 'x',
              'y': [suicide samuel  , committed suicide  , suicide  , suicide
                    related  , suicide article  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4781275987625122, 0.48078110814094543, 0.5196993947029114,
                    0.5256776809692383, 0.5358508229255676],
              'xaxis': 'x2',
              'y': [cause suicide  , committed suicide  , reason suicide  ,
                    suicide related  , suicide mistakenly  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.49354517459869385, 0.5052456259727478, 0.5190634727478027,
                    0.5281485319137573, 0.6453685164451599],
              'xaxis': 'x3',
              'y': [homeless  , suicide falling  , mental health  , suicides  ,
                    becoming homeless  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4489786624908447, 0.4668211042881012, 0.47294023633003235,
                    0.49355894327163696, 0.4982725977897644],
              'xaxis': 'x4',
              'y': [suicide walmart  , supervisor shot  , walmart manager  ,
                    walmart supervisor  , employee walmart  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.3973943889141083, 0.42890143394470215, 0.4903162121772766,
                    0.4933518171310425, 0.5451695919036865],
              'xaxis': 'x5',
              'y': [investigation  , pastor killed  , killed investigation  ,
                    sennett victim  , murder investigation  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.47624945640563965, 0.4942536950111389, 0.49859654903411865,
                    0.5026763677597046, 0.5464041233062744],
              'xaxis': 'x6',
              'y': [financial stress  , depression  , suffering depression  ,
                    depression financial  , mental health  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.3692171573638916, 0.37475836277008057, 0.42126816511154175,
                    0.42876625061035156, 0.49153125286102295],
              'xaxis': 'x7',
              'y': [problem category  , violating court  , legal problem  ,
                    problem article  , financial issues  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.42089951038360596, 0.43228885531425476, 0.45123761892318726,
                    0.4575120210647583, 0.5012936592102051],
              'xaxis': 'x8',
              'y': [cfo bed  , death company  , suicide  , arnal suicide  , jumped
                    death  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5046292543411255, 0.5208760499954224, 0.555534303188324,
                    0.5721696615219116, 0.6213339567184448],
              'xaxis': 'x9',
              'y': [panya kamrap  , killing panya  , perpetrator massacre  , panya
                    killed  , perpetrator panya  ],
              'yaxis': 'y9'},
      

In [322]:
model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0,
                                    'suicide article | suicide related | suicide | committed suicide | job related',
                                    270],
                                   [1,
                                    'becoming homeless | mental health | suicides | suicide falling | problem category',
                                    74],
                                   [2,
                                    'employee walmart | walmart supervisor | supervisor shot | walmart manager | suicide walmart',
                                    42],
                                   [3,
                                    'suicide related | suicide | jumped death | mentions cfo | company facing',
                                    34],
                                   [4,
                                    'financial crimes | legal problems | murdaugh family | lawsuit | article mentions',
                                    25],
                                   [5,
                                    'murder investigation | sennett victim | killed investigation | legal investigation | article',
                                    25],
                                   [6,
                                    'suicide mistakenly | robinhood trading | suicide related | mentions suicide | suicide due',
                                    24],
                                   [7,
                                    'perpetrator panya | drug charge | drug offense | mentions perpetrator | former police',
                                    18],
                                   [8,
                                    'rising suicides | suicides believed | suicides related | economic hardships | suicides',
                                    16],
                                   [9,
                                    'farm crisis | concerns suicides | farmers montana | rural communities | farmers article',
                                    14],
                                   [10,
                                    'financial problems | facing financial | meltdown article | problem financial | financial problem',
                                    13],
                                   [11,
                                    'life scam | ensnared phishing | phishing | phishing scheme | suicide article',
                                    11],
                                   [12,
                                    'cbp suicides | suicide context | suicides | mental health | suicide',
                                    9],
                                   [13,
                                    'financial scandal | madoff involved | suicide related | disgraced businessman | jeffrey epstein',
                                    9],
                                   [14,
                                    'divorce facing | financial problems | facing foreclosure | husband arrested | protection order',
                                    8],
                                   [15,
                                    'instagram scammers | instagram scammer | income instagram | income hackers | scammers hackers',
                                    8],
                                   [16,
                                    'paddock gunman | upset casinos | casinos | casinos treating | professional gambler',
                                    8],
                                   [17,
                                    'gambling problems | problem gambling | gambling addiction | sports gambling | mentions gambling',
                                    7],
                                   [18,
                                    'mental health | suffering depression | depression financial | depression | financial stress',
                                    7]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><b

In [323]:
# embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
# model.save("safetensors/financial4/", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

# # Method 2 - pytorch
# model.save("pytorch/financial4/", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# # Method 3 - pickle
# model.save("financial4", serialization="pickle")

2024-04-05 09:25:00,290 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [30]:
embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
model = BERTopic.load("financial4", embedding_model=embedding_model)

In [31]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,188,-1_suicide article_suicide related_suicide_com...,NaN,[The article mentions person committed suicide...
1,0,270,0_suicide article_suicide related_suicide_comm...,"[suicide article, suicide related, suicide, co...",[The article mentions person committed suicide...
2,1,74,1_becoming homeless_mental health_suicides_sui...,"[becoming homeless, mental health, suicides, s...",[The article mentions man considering suicide ...
3,2,42,2_employee walmart_walmart supervisor_supervis...,"[employee walmart, walmart supervisor, supervi...",[The article mentions Andre Bing Walmart super...
4,3,34,3_suicide related_suicide_jumped death_mention...,"[suicide related, suicide, jumped death, menti...",[The article mentions CFO Bed Bath Beyond jump...
5,4,25,4_financial crimes_legal problems_murdaugh fam...,"[financial crimes, legal problems, murdaugh fa...",[The article mentions Alex Murdaugh stealing m...
6,5,25,5_murder investigation_sennett victim_killed i...,"[murder investigation, sennett victim, killed ...",[The article mentions Charles Sennett victim\u...
7,6,24,6_suicide mistakenly_robinhood trading_suicide...,"[suicide mistakenly, robinhood trading, suicid...",[The article mentions Alexander Kearns 20-year...
8,7,18,7_perpetrator panya_drug charge_drug offense_m...,"[perpetrator panya, drug charge, drug offense,...",[The article mentions attacker Panya Kamrap fo...
9,8,16,8_rising suicides_suicides believed_suicides r...,"[rising suicides, suicides believed, suicides ...",[The article mentions rising suicides believed...


In [12]:
topics = model.topics_
probs = model.probabilities_

# from existing predictions
left = data['Left'].to_list()
indsleft = []
for i in range(len(comments_all)):
    if left[i] == 1:
        indsleft.append(i)

alltopicsleft = [topics[i] for i in indsleft if i < len(topics)]
allprobsleft = [probs[i] for i in indsleft if i < len(probs)]
allcommentsleft = [comments_all[i] for i in indsleft if i < len(comments_all)]
onetextleft = " ".join(allcommentsleft)

right = data['Right'].to_list()
indsright = []
for i in range(len(comments_all)):
    if right[i] == 1:
        indsright.append(i)

alltopicsright = [topics[i] for i in indsright if i < len(topics)]
allprobsright = [probs[i] for i in indsright if i < len(probs)]
allcommentsright = [comments_all[i] for i in indsright if i < len(comments_all)]
onetextright = " ".join(allcommentsright)

regional = data['Regional'].to_list()
indsreg = []
for i in range(len(comments_all)):
    if regional[i] == 1:
        indsreg.append(i)

alltopicsreg = [topics[i] for i in indsreg if i < len(topics)]
allprobsreg = [probs[i] for i in indsreg if i < len(probs)]
allcommentsreg = [comments_all[i] for i in indsreg if i < len(comments_all)]
onetextreg = " ".join(allcommentsreg)

national = data['National'].to_list()
indsnat = []
for i in range(len(comments_all)):
    if national[i] == 1:
        indsnat.append(i)

alltopicsnat = [topics[i] for i in indsnat if i < len(topics)]
allprobsnat = [probs[i] for i in indsnat if i < len(probs)]
allcommentsnat = [comments_all[i] for i in indsnat if i < len(comments_all)]
onetextnat = " ".join(allcommentsnat)

embeddingsleft = sentence_model.encode(onetextleft, show_progress_bar=True)
embeddingsright = sentence_model.encode(onetextright, show_progress_bar=True)
embeddingsnat = sentence_model.encode(onetextnat, show_progress_bar=True)
embeddingsreg = sentence_model.encode(onetextreg, show_progress_bar=True)


onetopicleft, oneprobleft = model.transform(onetextleft)
onetopicright, oneprobright = model.transform(onetextright)
onetopicreg, oneprobreg = model.transform(onetextreg)
onetopicnat, oneprobnat = model.transform(onetextnat)

topicwiseprobleft = np.array(allprobsleft).mean(axis = 0)
topicwiseprobright = np.array(allprobsright).mean(axis = 0)
topicwiseprobnat = np.array(allprobsnat).mean(axis = 0)
topicwiseprobreg = np.array(allprobsreg).mean(axis = 0)

np.set_printoptions(suppress=True, formatter={'float': '{:0.5f}'.format})
print("Individual averaged: ", np.round(topicwiseprobleft - topicwiseprobright, 5))
indavgpol = np.round(topicwiseprobleft - topicwiseprobright, 5)
print("Combined 1 text: ", np.round(oneprobleft - oneprobright, 5))
combpol = np.round(oneprobleft - oneprobright, 5)

print("Individual averaged: ", np.round(topicwiseprobnat - topicwiseprobreg, 5))
print("Combined 1 text: ", np.round(oneprobnat - oneprobreg, 5))
indavgreg = np.round(topicwiseprobnat - topicwiseprobreg, 5)
combreg = np.round(oneprobnat - oneprobreg, 5)

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.28it/s]


Individual averaged:  [0.01317 -0.01928 -0.00447 -0.03535 -0.00429 -0.00425 0.02081 0.00324
 0.00896 0.00345 -0.01044 -0.03897 0.00430 0.01947 -0.00256 0.02301
 0.00227 -0.01822 0.02292]
Combined 1 text:  [[0.02025 -0.05132 -0.00082 -0.00928 -0.00482 -0.00429 -0.00041 -0.00249
  -0.09429 0.01955 -0.00550 -0.00131 -0.00121 -0.00453 -0.00261 -0.00224
  -0.00067 -0.00036 0.00777]]
Individual averaged:  [-0.07306 -0.02307 0.00568 0.00987 0.01062 0.01318 0.02043 -0.02596
 0.01303 0.00265 -0.00241 -0.00572 0.00932 0.01426 0.00616 0.01671
 -0.00823 -0.00729 0.01570]
Combined 1 text:  [[-0.00738 0.05036 0.00001 -0.01823 0.00007 0.00008 0.00000 0.00007
  0.03831 0.02119 -0.01108 0.00002 0.00001 -0.00871 0.00004 0.00003
  0.00000 0.00000 0.02399]]


In [40]:
ben = data.copy()
probabilities = model.probabilities_
print(probabilities.shape)
for i in range(probabilities.shape[1]):
    ben['Topic' + str(i)] = probabilities[:, i]
ben.columns


(810, 19)


In [42]:
ben = ben.drop(columns=['Unnamed: 0', 'Reasoning'])

In [43]:
# ben = ben.rename(columns={FinancialPredicted: FinancialLabel, LegalPredicted: Legallabel, SchoolPredicted : SchoolLabel, HealthPredicted : HealthLabel})

In [44]:
ben.columns

Index(['id', 'title', 'content', 'Publisher', 'Date Published',
       'Financial/Job Problem', 'School or Academic Related Problem',
       'Lack of Access to Health/Mental Health Care', 'Legal Problem', 'None',
       'Only Reason', 'National', 'Regional', 'Left', 'Right',
       'Fringe/Questionable', 'Topic0', 'Topic1', 'Topic2', 'Topic3', 'Topic4',
       'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11',
       'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17',
       'Topic18'],
      dtype='object')

In [45]:
ben.to_csv('Financial problem articles topic model results.csv')

In [13]:
df = model.get_topic_info()

topicwiseprobl = np.insert(topicwiseprobleft, 0,-1)
topicwiseprobr = np.insert(topicwiseprobright, 0,-1)
topicwiseprobnat = np.insert(topicwiseprobnat, 0,-1)
topicwiseprobreg = np.insert(topicwiseprobreg, 0,-1)
df['Left - Right Individual Averaged'] = topicwiseprobl
df['Right - Individual Averaged'] = topicwiseprobr
df['National - Individual Averaged'] = topicwiseprobnat
df['Regional - Individual Averaged'] = topicwiseprobreg

oneprobl = np.insert(oneprobleft, 0,-1)
oneprobr = np.insert(oneprobright, 0,-1)
oneprobn = np.insert(oneprobnat, 0,-1)
oneprobre = np.insert(oneprobreg, 0,-1)
df['Left - One combined text'] = oneprobl
df['Right - One combined text'] = oneprobr
df['National - One combined text'] = oneprobn
df['Regional - One combined text'] = oneprobre

df['Left v/s Right - Individual Averaged'] = topicwiseprobl - topicwiseprobr
df['National v/s Regional - Individual Averaged'] = topicwiseprobnat - topicwiseprobreg
df['Left v/s Right - Combined Text'] = oneprobl - oneprobr
df['National v/s Regional - Combined Text'] = oneprobn - oneprobre

df.to_csv('financial betopic results.csv')

In [24]:
leftarticles = data[data['Left'] == 1]
rightarticles = data[data['Right'] == 1]
leftcontent = leftarticles['Only Reason'].to_list()
rightcontent = rightarticles['Only Reason'].to_list()
embeddingsleft = sentence_model.encode(leftcontent, show_progress_bar=True)
embeddingsright = sentence_model.encode(rightcontent, show_progress_bar=True)

topicsleft1, probsleft1 = model.transform(leftcontent, embeddingsleft)
topicsright1, probsright1 = model.transform(rightcontent, embeddingsright)

print(np.round(np.array(probsleft1).mean(axis = 0) - np.array(probsright1).mean(axis = 0), 5))

Batches: 100%|██████████| 6/6 [00:05<00:00,  1.07it/s]


[-0.01768 0.00186 0.01267 -0.02995 -0.00014 -0.00019 0.00768 -0.00013
 0.00858 0.00713 -0.00344 -0.00309 0.00485 0.01242 -0.00232 0.01456
 0.00053 -0.01802 0.01597]


In [25]:
count = 0
for i in range(len(alltopicsleft)):
    if alltopicsleft[i] != topicsleft1[i]:
        count += 1
print(count)

93
